<a href="https://colab.research.google.com/github/Amanuel94/kaggle/blob/main/CNN-transfer-learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
train_ds = tfds.load("tf_flowers", split ="train[:75%]", as_supervised = True)
val_ds = tfds.load("tf_flowers", split = "train[75%:90%]", as_supervised = True)
test_ds = tfds.load("tf_flowers", split = "train[90%:]", as_supervised = True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
 def preprocess(image, label):
  resized = tf.image.resize(image, [224, 224])
  final_image = tf.keras.applications.xception.preprocess_input(resized)
  return final_image, label

In [4]:
n_batch = 32
train_set = train_ds.map(preprocess).batch(n_batch).prefetch(1)
val_set = val_ds.map(preprocess).batch(n_batch).prefetch(1)
test_set = test_ds.map(preprocess).batch(n_batch).prefetch(1)

In [5]:
base_model = tf.keras.applications.xception.Xception(weights = "imagenet",
                                                   include_top = False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5, activation = "softmax")(avg)
model = tf.keras.Model(inputs = base_model.input, outputs = output)


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [6]:
for layer in base_model.layers:
  layer.trainable = False

In [7]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(
        learning_rate = 0.2,
        momentum = 0.9,
        decay = 0.01
    ),
    loss  = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)
model.fit(train_set, epochs = 5, validation_data = val_set)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


86/86 ━━━━━━━━━━━━━━━━━━━━ 36s 221ms/step - accuracy: 0.6600 - loss: 2.6691 - val_accuracy: 0.8584 - val_loss: 1.2675
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.8685 - loss: 1.1536 - val_accuracy: 0.8730 - val_loss: 1.0158
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 145ms/step - accuracy: 0.8921 - loss: 0.8030 - val_accuracy: 0.8675 - val_loss: 1.3200
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 147ms/step - accuracy: 0.9166 - loss: 0.5636 - val_accuracy: 0.8766 - val_loss: 1.1468
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.9253 - loss: 0.4661 - val_accuracy: 0.8947 - val_loss: 1.0506


In [8]:
for layer in base_model.layers:
  layer.trainable = True

model.compile(
  optimizer = tf.keras.optimizers.SGD(
      learning_rate = 0.01,
      momentum = 0.9,
      decay = 0.001
  ),
  loss  = "sparse_categorical_crossentropy",
  metrics = ["accuracy"]
)

In [9]:
model.fit(train_set, epochs = 5, validation_data = val_set)

Epoch 1/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 80s 516ms/step - accuracy: 0.8087 - loss: 0.6897 - val_accuracy: 0.8784 - val_loss: 0.4845
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 57s 495ms/step - accuracy: 0.9609 - loss: 0.1202 - val_accuracy: 0.9220 - val_loss: 0.3137
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 83s 508ms/step - accuracy: 0.9972 - loss: 0.0156 - val_accuracy: 0.9292 - val_loss: 0.3072
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 82s 511ms/step - accuracy: 0.9991 - loss: 0.0017 - val_accuracy: 0.9328 - val_loss: 0.3505
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 490ms/step - accuracy: 1.0000 - loss: 1.8194e-04 - val_accuracy: 0.9292 - val_loss: 0.3651


In [10]:
model.evaluate(test_set)

12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 729ms/step - accuracy: 0.9495 - loss: 0.1783


[0.16411742568016052, 0.9564032554626465]